<a href="https://colab.research.google.com/github/mavenceslau/Challenge-Data-Science-Alura-Cash/blob/master/notebooks/Notebook_01_Limpeza_de_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semana 02 - Aprendendo com os dados: criando um modelo de previsão de inadimplência

Após os tratamentos adequados no banco de dados, foi gerado um arquivo .csv, com ele continuaremos nossas análises mais minuciosas com auxílio da linguagem de programação Python.

Importando bibliotecas

In [1]:
import sys
sys.path.append('../')

import pandas as pd
import seaborn as sns
from utils import config_exibicao, config_graficos

Com a função **config_exibicao** iremos configurar a exibição de dataframes pandas, de modo que os mesmos possam exibir o maior número de informações nas células sem cortar conteudo.

E com a função **config_graficos** vamos configurar a exibição dos gráficos.

Mais informações sobre essas funções podem ser vistas no arquivo [utils.py]()

In [2]:
config_exibicao()
config_graficos()

Importando a base de dados e exibindo os primeiros registros

In [3]:
dados = pd.read_csv('../data/interim/alura_cash.csv')
dados.head()

,pessoa_idade,pessoa_salario_anual,pessoa_status_propriedade,pessoa_tempo_trabalho,emprestimo_motivo,emprestimo_pontuacao,emprestimo_valor_total,emprestimo_taxa_juros,emprestimo_inadimplente,emprestimo_renda_percentual,historico_inadimplencia,historico_tempo_credito
0,22.0,50000.0,Hipotecada,0.0,Pagamento de débitos,A,12800.0,5.79,0.0,0.26,N,2.0
1,26.0,84996.0,Alugada,0.0,Médico,B,9000.0,11.26,0.0,0.11,N,2.0
2,24.0,55782.0,Alugada,3.0,Melhora do lar,A,17000.0,7.51,0.0,0.30,N,2.0
3,25.0,125000.0,Própria,9.0,Pessoal,B,25000.0,10.59,0.0,0.20,N,2.0
4,23.0,44340.0,Própria,0.0,Empreendimento,C,3000.0,12.98,0.0,0.07,S,3.0


In [4]:
print(f'O dataset possui {dados.shape[0]} linhas e {dados.shape[1]} colunas')

O dataset possui 34501 linhas e 12 colunas


## Analisando os dados

In [5]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34501 entries, 0 to 34500
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   pessoa_idade                 34177 non-null  float64
 1   pessoa_salario_anual         34162 non-null  float64
 2   pessoa_status_propriedade    34170 non-null  object 
 3   pessoa_tempo_trabalho        33247 non-null  float64
 4   emprestimo_motivo            34186 non-null  object 
 5   emprestimo_pontuacao         34188 non-null  object 
 6   emprestimo_valor_total       34170 non-null  float64
 7   emprestimo_taxa_juros        30871 non-null  float64
 8   emprestimo_inadimplente      34158 non-null  float64
 9   emprestimo_renda_percentual  34182 non-null  float64
 10  historico_inadimplencia      34131 non-null  object 
 11  historico_tempo_credito      34497 non-null  float64
dtypes: float64(8), object(4)
memory usage: 3.2+ MB


As colunas **pessoa_idade**, **pessoa_tempo_trabalho** e **emprestimo_inadimplente** por algum motivo durante a exportação dos dados não está como tipo inteiro, logo irei fazer a conversão dos tipos, mas antes devemos verificar se há dados nulos no dataset

In [6]:
"""valores_int = ['pessoa_idade', 'pessoa_tempo_trabalho', 'emprestimo_inadimplente']

for valor in valores_int:
    dados[valor] = dados[valor].astype('int')"""

"valores_int = ['pessoa_idade', 'pessoa_tempo_trabalho', 'emprestimo_inadimplente']\n\nfor valor in valores_int:\n    dados[valor] = dados[valor].astype('int')"

In [7]:
descricao = dados.describe().T
descricao.style.background_gradient(cmap = 'crest').format(precision = 2)

,count,mean,std,min,25%,50%,75%,max
pessoa_idade,34177.00,27.73,6.35,20.00,23.00,26.00,30.00,144.00
pessoa_salario_anual,34162.00,66028.69,61405.05,4000.00,38493.00,55000.00,79200.00,6000000.00
pessoa_tempo_trabalho,33247.00,4.79,4.14,0.00,2.00,4.00,7.00,123.00
emprestimo_valor_total,34170.00,9590.58,6320.43,500.00,5000.00,8000.00,12200.00,35000.00
emprestimo_taxa_juros,30871.00,11.01,3.24,5.42,7.90,10.99,13.47,23.22
emprestimo_inadimplente,34158.00,0.22,0.41,0.00,0.00,0.00,0.00,1.00
emprestimo_renda_percentual,34182.00,0.17,0.11,0.00,0.09,0.15,0.23,0.83
historico_tempo_credito,34497.00,5.81,4.06,2.00,3.00,4.00,8.00,30.00


### Verifcando a existência de dados nulos

In [8]:
dados.isnull().sum()

pessoa_idade                    324
pessoa_salario_anual            339
pessoa_status_propriedade       331
pessoa_tempo_trabalho          1254
emprestimo_motivo               315
emprestimo_pontuacao            313
emprestimo_valor_total          331
emprestimo_taxa_juros          3630
emprestimo_inadimplente         343
emprestimo_renda_percentual     319
historico_inadimplencia         370
historico_tempo_credito           4
dtype: int64

Podemos ver que existem muitos dados nulos no nosso dataset. Existem algumas maneiras de resolver essa questão. 

Uma estratégia seria remover todos os dados nulos, que ao meu ver não é recomendado, pois haveria muita perda de dados. 

Outra abordagem seria no lugar dos dados nulos, preencher com a informação da média da coluna correspondente. Irei usar esse método.

Para as colunas numéricas usarei as médias e para as colunas categóricas usarei o dado mais frequente.

In [9]:
# Substituindo os dados faltantes das colunas numéricas pelas médias de cada coluna

numericas = ['pessoa_idade', 'pessoa_salario_anual', 'pessoa_tempo_trabalho', 'emprestimo_valor_total',
             'emprestimo_taxa_juros', 'emprestimo_renda_percentual', 'historico_tempo_credito']

for numero in numericas:
       dados[numero].fillna(dados[numero].mean(), inplace = True)

A coluna **emprestimo_inadimplente** é nossa coluna alvo(target) para o modelo de machine learning, por isso os dados nulos da mesma serão removidos

In [10]:
dados.dropna(subset = 'emprestimo_inadimplente', inplace = True)

Exibindo a frequencia de registros das variáveis categóricas

In [11]:
categorias = ['pessoa_status_propriedade', 'emprestimo_motivo', 'emprestimo_pontuacao', 'historico_inadimplencia']

for cat in categorias:
    print(25*'--')
    print(dados[cat].value_counts())
    print(25*'--')

--------------------------------------------------
Alugada       17080
Hipotecada    13983
Própria        2689
Outros          109
Name: pessoa_status_propriedade, dtype: int64
--------------------------------------------------
--------------------------------------------------
Educativo               6702
Médico                  6317
Empreendimento          5932
Pessoal                 5729
Pagamento de débitos    5424
Melhora do lar          3757
Name: emprestimo_motivo, dtype: int64
--------------------------------------------------
--------------------------------------------------
A    11224
B    10849
C     6710
D     3767
E     1001
F      252
G       64
Name: emprestimo_pontuacao, dtype: int64
--------------------------------------------------
--------------------------------------------------
N    27866
S     5960
Name: historico_inadimplencia, dtype: int64
--------------------------------------------------


In [12]:
# Substituindo os dados faltantes das colunas categóricas pelos valores mais comuns de cada coluna

categorias = ['pessoa_status_propriedade', 'emprestimo_motivo', 'emprestimo_pontuacao', 'historico_inadimplencia']

for cat in categorias:
    dados[cat].fillna(dados[cat].value_counts()[0], inplace = True)

Verificando a existência de dados nulos após os procedimentos

In [13]:
dados.isnull().sum()

pessoa_idade                   0
pessoa_salario_anual           0
pessoa_status_propriedade      0
pessoa_tempo_trabalho          0
emprestimo_motivo              0
emprestimo_pontuacao           0
emprestimo_valor_total         0
emprestimo_taxa_juros          0
emprestimo_inadimplente        0
emprestimo_renda_percentual    0
historico_inadimplencia        0
historico_tempo_credito        0
dtype: int64

### Verifcando a existência de dados duplicados

In [14]:
dados.duplicated().value_counts()
print(f'Existem {dados.duplicated().value_counts()[1]} registros duplicados no dataset')

Existem 165 registros duplicados no dataset


In [15]:
dados.drop_duplicates(inplace = True)

In [16]:
dados.duplicated().value_counts()

False    33993
dtype: int64

### Verifcando a existência de outliers